In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# 학습용 데이터셋 불러오기
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/Colab/Kaggle_translated.xlsx', names = ["sentiment","english", "korean" ])
df.head()

,sentiment,english,korean
0,neutral,Technopolis plans to develop in stages an area...,"테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 100,0..."
1,negative,The international electronic industry company ...,국제적인 전자 산업 회사 Elcoteq는 그것의 Talinn 시설로부터 10명의 직...
2,positive,With the new production plant the company woul...,새로운 생산 공장과 함께 회사는 수요의 예상되는 증가를 충족시키기 위하여 그것의 능...
3,positive,According to the company 's updated strategy f...,"2009-2012년 회사의 최신 전략에 따르면, Basware는 순매출의 10%-2..."
4,positive,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...,ASPOCOMP의 파이낸싱은 기술적으로 더 까다로운 HDI 인쇄 회로 기판 PCB에...


# 데이터 라벨링

In [4]:
import numpy as np

In [5]:
labels = []
for sentiment in df['sentiment']:
  if (sentiment == 'neutral'):
    labels.append(2)
  if (sentiment == 'negative'):
    labels.append(0)
  if (sentiment == 'positive'):
    labels.append(1)

df['label'] = labels
  

In [6]:
# 라벨링이 0,1,2로 잘 되었음을 확인
df['label'].unique()

array([2, 0, 1])

In [7]:
df

,sentiment,english,korean,label
0,neutral,Technopolis plans to develop in stages an area...,"테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 100,0...",2
1,negative,The international electronic industry company ...,국제적인 전자 산업 회사 Elcoteq는 그것의 Talinn 시설로부터 10명의 직...,0
2,positive,With the new production plant the company woul...,새로운 생산 공장과 함께 회사는 수요의 예상되는 증가를 충족시키기 위하여 그것의 능...,1
3,positive,According to the company 's updated strategy f...,"2009-2012년 회사의 최신 전략에 따르면, Basware는 순매출의 10%-2...",1
4,positive,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...,ASPOCOMP의 파이낸싱은 기술적으로 더 까다로운 HDI 인쇄 회로 기판 PCB에...,1
...,...,...,...,...
4840,negative,LONDON MarketWatch -- Share prices ended lower...,런던 마켓워치 -- FTSE 100에 대한 은행 주식의 반등이 더 큰 약세를 상쇄하...,0
4841,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...,린쿠스키아의 맥주 판매량은 416만 리터로 6.5퍼센트 감소했고 카우노 알루스의 맥...,2
4842,negative,Operating profit fell to EUR 35.4 mn from EUR ...,"영업 이익은 EUR 12.3 mn의 선박 판매 이익을 포함하여, 2007년에 EUR...",0
4843,negative,Net sales of the Paper segment decreased to EU...,종이 부문의 순매출은 2008년 2/4분기에 EUR 241.1 mn에서 2009년 ...,0


In [8]:
data_list = []
for q, label in zip(df['korean'], df['label'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [9]:
print(len(data_list))
print(data_list[0])
print(data_list[2000])
print(data_list[-1])

4845
['테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 100,000 평방미터 이상의 면적을 단계적으로 개발할 계획이라고 성명은 말했다.', '2']
['A & 유로; Lewa의 13.6%에 480만 투자; Dutsche Beteiligungs & 유로; 2100만 달러 순이익.', '1']
['핀란드에서 판매는 1월에 10.5% 감소하였습니다, 반면에 핀란드 밖에서 판매는 17%까지 떨어졌습니다.', '0']


In [10]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)
print(len(dataset_train))
print(len(dataset_test))
print(dataset_train[0])
print(dataset_test[0])

3633
1212
['그러나 2010년 4월부터 6월까지 매출이 성장세로 돌아섰다고 CEO Pekka Eloholma는 말했다.', '1']
["주문은 중앙 스웨덴, Plefa's Hassela 밀을 위한 파켓 베이스 레이어 베니어 생산을 위한 통나무 취급 라인, 벗겨지는 라인 그리고 건조 라인을 포함합니다.", '2']


## 모델 불러오기

In [11]:
# Hugging Face의 트랜스포머 모델을 설치
!pip install transformers

     |████████████████████████████████| 3.3 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 49.9 MB/s 
     |████████████████████████████████| 3.3 MB 27.7 MB/s 
     |████████████████████████████████| 61 kB 499 kB/s 
     |████████████████████████████████| 895 kB 49.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [12]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 46.9 MB 2.4 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 5.2 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595728 sha256=b5880c392aac884be784f9fe4f98b128dbf80c52d545614a9e341c407ce78ccb
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 5.4 MB/s 
     |████████████████████████████████| 769 kB 5.1 MB/s 
     |████████████████████████████████| 3.0 MB 33.9 MB/s 
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.3
    Uninstalling tokenizers-0.10.3:
      Successfully uninstalled tokenizers-0.10.3
  Attempting uninstall: transformers
    Found existing installation: transfo

In [13]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-n4jcwi_7
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-n4jcwi_7
     |████████████████████████████████| 4.9 MB 5.2 MB/s 
  Using cached transformers-4.13.0-py3-none-any.whl (3.3 MB)
  Using cached tokenizers-0.10.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (3.3 MB)
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=13136 sha256=2ddfc2fdc2d379ef15bda38fbb22c4655f64f22832528ab4a15453ec69811661
  Stored in directory: /tmp/pip-ephem-wheel-cache-7l2nx1bh/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.8.1rc1
    Uninstalling tokenizers-0.8.1rc1:
      Successfully uninstalled tokenizers-0.8.1rc1
  Attempting uninstall: transformers
    Found existi

In [14]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [15]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [16]:
#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

.cache/kobert_v1.zip[██████████████████████████████████████████████████]
.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


# 모델 학습 

In [17]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [18]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [19]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [20]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [21]:
data_train[0]

(array([   2, 1199,  567, 5712,  605, 7033,  617, 7029, 2003, 2794, 6079,
        1734, 6572, 5782, 5439,  640,  681,  389,  415,  415,  367,  645,
         422,  403,  420,  423,  367, 5760, 1966,  517,   54,    3,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(32, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 1)

In [22]:
data_test[1]

(array([   2,  635,  420,  423,  367,  671,  390,  405,  367, 6983,  635,
         437,  425,  367,  681,  375,  442,  425,  394,  440, 7095, 3017,
        5937,  774, 7095, 4329, 6903,  635,  420,  423,  367, 7095,  517,
        6147, 7765, 6664, 2725, 3726, 7789, 3566, 5073,  986, 4492, 7836,
         919,  517,   54,    3,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(48, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 2)

In [23]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [24]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [25]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [26]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [27]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [28]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [29]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [30]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [31]:
train_dataloader


# 학습

In [32]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/57 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.0284368991851807 train acc 0.578125
epoch 1 train acc 0.5915458288578589


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 9 test acc 0.8373903508771929


  0%|          | 0/57 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.00542466202750802 train acc 1.0
epoch 10 train acc 0.9961622807017544


  0%|          | 0/19 [00:00<?, ?it/s]

epoch 10 test acc 0.8391447368421052


# 모델 저장

In [34]:
import os
os.chdir('/content/drive/MyDrive/Colab/models')
os.getcwd()

'/content/drive/MyDrive/Colab/models'

In [35]:
path = '/content/drive/MyDrive/Colab/models/'
torch.save(model, path + 'financial_3_class.pt')  # 전체 모델 저장

In [39]:
torch.save(model.state_dict(), 'financial_3_class_state_dict.pt')  # 모델 객체의 state_dict 저장

In [40]:
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, 'financial_3_class_all.tar')  # 여러 가지 값 저장, 학습 중 진행 상황 저장을 위해 epoch, loss 값 등 일반 scalar값 저장 가능

# 모델 불러오기

In [41]:
import os
os.chdir('/content/drive/MyDrive/Colab/models/')

model1 = torch.load('financial_3_class.pt')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수
model1.load_state_dict(torch.load('financial_3_class_state_dict.pt'))  # state_dict를 불러 온 후, 모델에 저장

checkpoint = torch.load('financial_3_class_all.tar')   # dict 불러오기
model1.load_state_dict(checkpoint['model'])
optimizer.load_state_dict(checkpoint['optimizer'])

# 테스트하기

In [42]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [51]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model1.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model1(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("부정")
            elif np.argmax(logits) == 1:
                test_eval.append("긍정")
            elif np.argmax(logits) == 2:
                test_eval.append("중립")            

        print(test_eval[0] + " 문장입니다.")

In [52]:
for i in range(3):
    sentence = input("문장을 입력해주세요 : ")    
    predict(sentence)
    print("\n")

문장을 입력해주세요 : 최근 3년간 성장세가 주춤했으나 내년부터 LG그룹 주요 계열사들의 공장 증설 및 설비 투자 확대와 맞물려 약 1조5000억원 규모로 매출 성장이 이뤄질 거라고 자이에스앤디 측은 설명했다.


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


긍정 문장입니다.


문장을 입력해주세요 : 삼성전자가 올 3분기 반도체 매출에서 미국 인텔을 제치고 글로벌 1위에 올랐다는 조사 결과가 발표됐다
긍정 문장입니다.


문장을 입력해주세요 : 반면 인텔은 주력 제품인 MPU(마이크로프로세서유닛)의 매출이 거의 제자리걸음 하면서 전 분기 대비 1.8% 감소한 187억8600만달러의 매출을 기록해 2위로 내려앉았다
부정 문장입니다.


